In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


# Language Translation with ``nn.Transformer`` and torchtext

This tutorial shows:
    - How to train a translation model from scratch using Transformer.
    - Use torchtext library to access  [Multi30k](http://www.statmt.org/wmt16/multimodal-task.html#task1)_ dataset to train a German to English translation model.


## Data Sourcing and Processing

[torchtext library](https://pytorch.org/text/stable/)_ has utilities for creating datasets that can be easily
iterated through for the purposes of creating a language translation
model. In this example, we show how to use torchtext's inbuilt datasets,
tokenize a raw text sentence, build vocabulary, and numericalize tokens into tensor. We will use
[Multi30k dataset from torchtext library](https://pytorch.org/text/stable/datasets.html#multi30k)_
that yields a pair of source-target raw sentences.

To access torchtext datasets, please install torchdata following instructions at https://github.com/pytorch/data.




In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}

Create source and target language tokenizer. Make sure to install the dependencies.

```python
pip install -U torchdata
pip install -U spacy
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
```


In [3]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

## Seq2Seq Network using Transformer

Transformer is a Seq2Seq model introduced in [“Attention is all you
need”](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)_
paper for solving machine translation tasks.
Below, we will create a Seq2Seq network that uses Transformer. The network
consists of three parts. First part is the embedding layer. This layer converts tensor of input indices
into corresponding tensor of input embeddings. These embedding are further augmented with positional
encodings to provide position information of input tokens to the model. The second part is the
actual [Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)_ model.
Finally, the output of the Transformer model is passed through linear layer
that gives unnormalized probabilities for each token in the target language.




In [4]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

During training, we need a subsequent word mask that will prevent the model from looking into
the future words when making predictions. We will also need masks to hide
source and target padding tokens. Below, let's define a function that will take care of both.




In [5]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Let's now define the parameters of our model and instantiate the same. Below, we also
define our loss function which is the cross-entropy loss and the optimizer used for training.




In [7]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/home/steven/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## Collation

As seen in the ``Data Sourcing and Processing`` section, our data iterator yields a pair of raw strings.
We need to convert these string pairs into the batched tensors that can be processed by our ``Seq2Seq`` network
defined previously. Below we define our collate function that converts a batch of raw strings into batch tensors that
can be fed directly into our model.




In [8]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [9]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

Let's define training and evaluation loop that will be called for each
epoch.




Now we have all the ingredients to train our model. Let's do it!




In [10]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

/home/steven/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/home/steven/miniconda3/envs/torch/lib/python3.9/site-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 5.343, Val loss: 4.110, Epoch time = 15.455s
Epoch: 2, Train loss: 3.764, Val loss: 3.320, Epoch time = 14.545s
Epoch: 3, Train loss: 3.162, Val loss: 2.891, Epoch time = 14.717s
Epoch: 4, Train loss: 2.770, Val loss: 2.638, Epoch time = 14.720s
Epoch: 5, Train loss: 2.481, Val loss: 2.446, Epoch time = 14.680s
Epoch: 6, Train loss: 2.251, Val loss: 2.314, Epoch time = 14.795s
Epoch: 7, Train loss: 2.057, Val loss: 2.221, Epoch time = 14.741s
Epoch: 8, Train loss: 1.896, Val loss: 2.114, Epoch time = 14.732s
Epoch: 9, Train loss: 1.756, Val loss: 2.066, Epoch time = 14.741s
Epoch: 10, Train loss: 1.634, Val loss: 2.008, Epoch time = 14.828s
Epoch: 11, Train loss: 1.519, Val loss: 1.968, Epoch time = 14.757s
Epoch: 12, Train loss: 1.418, Val loss: 1.937, Epoch time = 14.742s
Epoch: 13, Train loss: 1.333, Val loss: 1.935, Epoch time = 14.815s
Epoch: 14, Train loss: 1.251, Val loss: 1.965, Epoch time = 14.722s
Epoch: 15, Train loss: 1.177, Val loss: 1.957, Epoch time

In [11]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A group of people standing in front of an igloo . 


In [12]:
print(translate(transformer, "Zwei Autos fahren auf einer Rennstrecke."))

 Two cars are racing around in a track . 


##  Evaluation on Test dataset

In [15]:
%ls -al ../../.data

total 5404
drwxrwxr-x  2 steven steven    4096 Jan  8 16:49 ./
drwxrwxr-x 11 steven steven    4096 Jan  8 16:48 ../
-rw-rw-r--  1 steven steven   70649 Jan  7 15:59 test_2016_flickr.de
-rw-rw-r--  1 steven steven   22890 Jan  7 15:22 test_2016_flickr.de.gz
-rw-r--r--  1 steven steven   16384 Jan  8 16:50 .test_2016_flickr.de.swp
-rw-rw-r--  1 steven steven   62076 Jan  7 15:59 test_2016_flickr.en
-rw-rw-r--  1 steven steven   21077 Jan  7 15:22 test_2016_flickr.en.gz
-rw-rw-r--  1 steven steven 2110398 Jan  7 15:59 train.de
-rw-rw-r--  1 steven steven  637044 Jan  7 15:21 train.de.gz
-rw-rw-r--  1 steven steven 1801238 Jan  7 15:59 train.en
-rw-rw-r--  1 steven steven  568929 Jan  7 15:21 train.en.gz
-rw-rw-r--  1 steven steven   75981 Jan  7 15:59 val.de
-rw-rw-r--  1 steven steven   24681 Jan  7 15:21 val.de.gz
-rw-rw-r--  1 steven steven   63297 Jan  7 15:59 val.en
-rw-rw-r--  1 steven steven   21650 Jan  7 15:21 val.en.gz


In [17]:
filepaths = ("../../.data/test_2016_flickr.de", "../../.data/test_2016_flickr.en")
def create_pairs(filepaths):
    pairs = []
    with open(filepaths[0], encoding="utf8") as f1, open(filepaths[1], encoding="utf8") as f2:
        pairs = [(line1, line2) for line1, line2 in zip(f1, f2)]
    return pairs
pairs = create_pairs(filepaths)

In [21]:
for i, (src_text, tgt_text) in enumerate(pairs):
    pred_text = translate(transformer, src_text)
    print(f"##### {i}")
    print(f"src : {src_text}\npred:{pred_text}\ngt  : {tgt_text}")

##### 0
src : Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.

pred: A man in an orange hat with something ponytail on . 
gt  : A man in an orange hat starring at something.

##### 1
src : Ein Boston Terrier läuft über saftig-grünes Gras vor einem weißen Zaun.

pred: A cute puppy runs over the grass in front of a white fence . 
gt  : A Boston Terrier is running on lush green grass in front of a white fence.

##### 2
src : Ein Mädchen in einem Karateanzug bricht ein Brett mit einem Tritt.

pred: A girl in a karate uniform is doing a board with a kick . 
gt  : A girl in karate uniform breaking a stick with a front kick.

##### 3
src : Fünf Leute in Winterjacken und mit Helmen stehen im Schnee mit Schneemobilen im Hintergrund.

pred: Five people in winter jackets and helmets stand in the snow with tattoos in the background . 
gt  : Five people wearing winter jackets and helmets stand in the snow, with snowmobiles in the background.

##### 4
src : Leute Reparieren das Dach eines

##### 41
src : Ein Mann in Uniform und ein Mann in einem blauen Hemd stehen vor einem Wagen.

pred: A man in a blue shirt and a man in a blue shirt are standing in front of a cart . 
gt  : A man in uniform and a man in a blue shirt are standing in front of a truck.

##### 42
src : Leute sitzen in einem Zug.

pred: People sit at a train station . 
gt  : People sit inside a train.

##### 43
src : Ein Junge schaukelt mit den Füßen in der Luft in einem Wald.

pred: A boy swings his feet in the air with his feet in a forest . 
gt  : A kid swings with his feet up in the air in a forest.

##### 44
src : Ein Mann in einem roten Hemd betritt ein Etablissement.

pred: A man in a red shirt is entering a giant weight performance . 
gt  : A man in a red shirt entering an establishment.

##### 45
src : Zwei Männer in Badehosen springen auf einem mäßig belebten Strand in die Luft.

pred: Two men in swim trunks are jumping in the air on a busy beach . 
gt  : Two men wearing swim trunks jump in the air

##### 81
src : Ein Mann in einem schwarzen T-Shirt spielt Trommel auf einem umgedrehten gelben Kübel.

pred: A man in a black t - shirt plays a drum made up at a yellow bucket . 
gt  : A man in a black t-shirt, cap and jeans is playing drums on an upside down yellow pail.

##### 82
src : Eine junge Künstlerin malt ein Bild einer Frau auf eine Wand.

pred: A young female performer is painting a picture on a wall . 
gt  : A young female artists paints an image of a woman on a wall.

##### 83
src : Zwei Spielerinnen der US-Nationalmannschaft klatschen im Sprung ab, umgeben von zwei weiteren Spielerinnen.

pred: Two military players are performing high - high jump in the high - uniforms are surrounded by two other other players .
gt  : Two female members of team USA performing a jump high-five surrounded by two other female members.

##### 84
src : Ein Mann rührt in einem Topf in seiner Küche.

pred: A man is stirring tea in his kitchen in his kitchen . 
gt  : A man stirring a pot of liqui

##### 119
src : Zwei braune Hunde rennen durch den Schnee.

pred: Two brown dogs running through the snow - covered snow . 
gt  : Two brown dogs are running through the snow.

##### 120
src : Bühnenfoto einer kleinen Band, die für ein Theaterpublikum spielt.

pred: A female band plays with a happy card game . 
gt  : On stage photo of small band performing for theater audience.

##### 121
src : Ein Baby im Weihnachtskostüm blickt in die Kamera.

pred: A baby in a vibrant bodysuit is looking at the camera . 
gt  : A baby in a Christmas outfit looks at the camera.

##### 122
src : Ein Mann Sitzt neben einem Computermonitor.

pred: A man is carving his ear - yo next to a rock . 
gt  : A man is sitting next to a computer monitor.

##### 123
src : Diese Frau hat gerade einen lustigen Witz gehört und lacht.

pred: This woman is about to be having a good card and laughing . 
gt  : This lady has heard a funny joke and laughing.

##### 124
src : Personen in einem Gebäude, eine von ihnen macht ei

##### 162
src : Zwei Hunde spielen an einem Baum.

pred: Two dogs play by a tree . 
gt  : Two dogs play by a tree.

##### 163
src : Ein Marineoffizier in einem schwarzen Mantel und Hose steigt Treppen während die Kamera auf den Hund fokussiert.

pred: A female athlete in a black coat and pants gets the camera while the camera takes a huge boulder . 
gt  : A navy walking wearing a black coat and pants on the stairs as camera focuses on the dog

##### 164
src : Zwei grün gekleidete Männer bereiten in einem Restaurant Essen zu.

pred: Two men in green prepare to take tea in a restaurant . 
gt  : Two men dressed in green are preparing food in a restaurant.

##### 165
src : Ein Mann in schwarzem Leder und einem Cowboyhut schlendert auf einem Renaissancefestival.

pred: A man in black leather coat and cowboy hat strolls by a stack of toddlers . 
gt  : A man dressed in black leather and a cowboy hat is walking around a Renaissance Festival.

##### 166
src : Ein Hund läuft mit einem gelben Spi

...

##### 845
src : Ein kleines Mädchen mit blonden Haaren spielt und spritzt in einer Schlammpfütze.

pred: A little girl with blond - hair plays in a mud puddle and splashing mud . 
gt  : A little girl with blond-hair is playing and splashing in a mud puddle.

##### 846
src : Ein Kind fährt auf einem Fahrrad durch eine Gasse mit einem Graffiti.

pred: A child rides a bicycle through a graffiti - covered alley . 
gt  : A child is riding a bike down an alleyway with graffiti in it.

##### 847
src : Eine Turnerin wird bei einer Veranstaltung beurteilt.

pred: A gymnast is being lifted at an event with female routine . 
gt  : A gymnast is being judged at en event.

##### 848
src : Menschen gehen auf einer Straße mit einem Straßenhändler.

pred: People walk on a street street with a street vendor . 
gt  : People walking down the street with a street vendor.

##### 849
src : Ein Mann in traditionellem Gewand steht neben seinem Esel, der anscheinend auch bekleidet ist.

pred: A man in tradition

##### 884
src : Eine Frau spielt mit Fingerpuppen während ein kleines Kind in einem Kostüm vorbeigeht.

pred: A woman is playing with a happy child in costume while passing by a small child in costume . 
gt  : A woman plays with finger puppets as a small child in a costume walks by.

##### 885
src : Eine Gruppe von Leuten beobachtet junge Männer, die auf Eimern Schlagzeug spielen.

pred: A group of young people watching young men play drums on buckets . 
gt  : A group of people watch young men play the drums using makeshift buckets as instruments.

##### 886
src : Ein Mädchen mit gewöhnlicher und improvisierter Schutzausrüstung fährt Rollerblades.

pred: A girl with matching bandanna and camouflage gear rides rollerblades . 
gt  : A girl with normal and improvised safety gear rollerblades.

##### 887
src : Ein kleines weißes Auto steht auf den Bahngleisen und wurde eventuell vom Zug im Hintergrund gerammt.

pred: A small white car stands on railroad tracks , but got just written in the

##### 917
src : Ein Mann mit einer Art Messsonde spricht mit einer Familie, die versucht, zu lächeln und freundlich zu sein.

pred: A man with some sort of dancers is talking to a family , trying to get his birthday party , while they are dancing . 
gt  : A man talking to a family unit holding some sort of probing device as they look on and smile to be polite.

##### 918
src : Sehr kleiner Junge in einem grünen Shirt liegt mit dem Gesicht nach unten auf einem weißen Bett.

pred: A large boy in a green shirt lies face down on a white bed . 
gt  : Very young boy in a green shirt laying face down on a white bed.

##### 919
src : Ein Extremradfahrer ruht sich aus während im Hintergrund die Sonne untergeht.

pred: A female athlete is resting in the sun , while wearing the sun sets . 
gt  : An extreme cyclist stops to rest as the sun sets in the background.

##### 920
src : Eine warm gekleidete Frau fährt in einem verschneiten Gebiet Ski.

pred: A woman dressed in a snowy area is skiing in a

##### 959
src : Ein Mann mit kariertem Hut in einer schwarzen Jacke und einer schwarz-weiß gestreiften Hose spielt auf einer Bühne mit einem Sänger und einem weiteren Gitarristen im Hintergrund auf einer E-Gitarre.

pred: A man in a plaid hat and black jacket and black striped pants plays on a stage with a singer in an electric guitar , with an electric guitar player in the background . 
gt  : A man in a black jacket and checkered hat wearing black and white striped pants plays an electric guitar on a stage with a singer and another guitar player in the background.

##### 960
src : Ein Mann springt von einem Stier weg über eine Absperrung.

pred: A man jumps off a bull over a tarp . 
gt  : A man is leaping over a barrier away from a bull.

##### 961
src : Eine Frau in einem blauen Shirt und weißen Shorts spielt Tennis.

pred: A woman in a blue shirt and white shorts is playing tennis . 
gt  : A woman in a blue shirt and white shorts playing tennis.

##### 962
src : Ein Mann spielt ein 

##### 991
src : Ein brauner Hund hebt einen Zweig von einer steinernen Oberfläche.

pred: A brown dog is lifting a branch from a stone surface . 
gt  : A brown dog picks up a twig from a stone surface.

##### 992
src : Das ist ein gelb gekleideter Mann, der ein braunes Pferd an der Leine hält.

pred: The man in yellow is holding a brown horse against the leash . 
gt  : This is a man dressed in yellow holding the reign of a brown horse

##### 993
src : Ein Mann in einem weißen Shirt und einer Schürze zerlegt einen Vogel.

pred: A man in a white shirt and apron cutting a bird bird . 
gt  : A man in a white shirt and apron cuts up a bird.

##### 994
src : Eine hispanische Frau kocht im Freien in einem Wok.

pred: A female athlete is cooking tricks in a cluttered studio . 
gt  : A hispanic woman is using a outdoors wok to cook.

##### 995
src : Marathonläuferinnen laufen auf einer städtischen Straße während andere Personen um sie herum stehen.

pred: Customers are walking around on a city 

## References

1. Attention is all you need paper.
   https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf
2. The annotated transformer. https://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding

